# Spotify Hit Prediction (Simple Notebook)

本 Notebook 是为 Data Science 期末项目准备的简化版流程示例，包含：
1. 读取并初步了解数据
2. 简单数据清洗与新建二分类目标（Hit / Not Hit）
3. 使用 Decision Tree 和 Random Forest 训练模型并评估

> 注意：这里使用的是已经下载好的 `spotify-2023.csv` 文件。

In [ ]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# 1. 读取数据（路径根据你本机的位置修改，如果和本示例不同的话）
file_path = "spotify-2023.csv"  # 如果在同一文件夹下，保持这样即可

# 有些 CSV 会有编码问题，这里指定 encoding，并忽略坏字符
df = pd.read_csv(file_path, encoding="latin1")

# 查看前 5 行
df.head()


## 1. 基本数据探索 (EDA)
这里我们简单看看数据的整体结构，包括：
- 行列数
- 列名
- 数值列的统计信息

In [ ]:

# 数据的行列数
print("Shape (rows, columns):", df.shape)

# 列名
print("\nColumns:\n", df.columns.tolist())

# 数值列的描述性统计
df.describe(include=[np.number]).T


## 2. 构造二分类目标：Hit vs Not Hit
在这个数据集中没有直接的 `popularity` 列，我们使用 `streams`（播放量）来定义 Hit：

- 计算 `streams` 的 75% 分位数（Q3）
- 如果某首歌的 `streams` ≥ Q3，则标记为 `Hit = 1`
- 否则为 `Hit = 0`


In [ ]:

# 确保 streams 列为数值类型
df['streams'] = pd.to_numeric(df['streams'], errors='coerce')

# 去掉 streams 为空的行
df = df.dropna(subset=['streams'])

# 使用 streams 分位数定义 Hit
q3 = df['streams'].quantile(0.75)
print("75% quantile of streams:", q3)

df['hit'] = (df['streams'] >= q3).astype(int)

# 看看 0 和 1 的分布
df['hit'].value_counts()


## 3. 简单可视化
我们画一个柱状图，看一下 Hit(1) 和 Not Hit(0) 的数量对比。

In [ ]:

hit_counts = df['hit'].value_counts().sort_index()

plt.figure(figsize=(4, 3))
plt.bar(hit_counts.index.astype(str), hit_counts.values)
plt.xlabel('Hit (1) vs Not Hit (0)')
plt.ylabel('Count')
plt.title('Distribution of Hit Label')
plt.tight_layout()
plt.show()


### 3.1 额外可视化：`bpm` 的直方图
再画一个简单的直方图，看看歌曲节奏（bpm）的分布情况。

In [ ]:

# 额外可视化：bpm 分布直方图
plt.figure(figsize=(5, 3))
df['bpm'].hist(bins=30)
plt.xlabel('BPM')
plt.ylabel('Count')
plt.title('Distribution of Song BPM')
plt.tight_layout()
plt.show()


## 4. 特征选择 (Features)
我们从数据中选取几个和歌曲特征相关的数值列作为特征：
- `bpm`
- `danceability_%`
- `valence_%`
- `energy_%`
- `acousticness_%`
- `instrumentalness_%`
- `liveness_%`
- `speechiness_%`

这些特征用于预测 `hit`。

In [ ]:

feature_cols = [
    'bpm',
    'danceability_%',
    'valence_%',
    'energy_%',
    'acousticness_%',
    'instrumentalness_%',
    'liveness_%',
    'speechiness_%'
]

# 把这些列转换成数值类型（防止有脏数据）
for col in feature_cols:
    df[col] = pd.to_numeric(df[col], errors='coerce')

# 丢弃在这些特征或 target 上有缺失值的行
model_df = df.dropna(subset=feature_cols + ['hit'])

X = model_df[feature_cols]
y = model_df['hit']

print("X shape:", X.shape)
print("y distribution:\n", y.value_counts())


## 5. 划分训练集与测试集，并训练模型
我们会做：
1. 使用 `train_test_split` 划分训练集和测试集（80% / 20%）
2. 训练一个 `DecisionTreeClassifier`
3. 训练一个 `RandomForestClassifier`
4. 比较它们在测试集上的 Accuracy 和混淆矩阵


In [ ]:

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print("Train size:", X_train.shape[0])
print("Test size:", X_test.shape[0])


In [ ]:

# ---- Decision Tree ----
dt_clf = DecisionTreeClassifier(random_state=42)
dt_clf.fit(X_train, y_train)

y_pred_dt = dt_clf.predict(X_test)

acc_dt = accuracy_score(y_test, y_pred_dt)
cm_dt = confusion_matrix(y_test, y_pred_dt)

print("Decision Tree Accuracy:", acc_dt)
print("Decision Tree Confusion Matrix:\n", cm_dt)
print("\nClassification Report (Decision Tree):\n")
print(classification_report(y_test, y_pred_dt))


In [ ]:

# ---- Random Forest ----
rf_clf = RandomForestClassifier(
    n_estimators=100,
    random_state=42
)
rf_clf.fit(X_train, y_train)

y_pred_rf = rf_clf.predict(X_test)

acc_rf = accuracy_score(y_test, y_pred_rf)
cm_rf = confusion_matrix(y_test, y_pred_rf)

print("Random Forest Accuracy:", acc_rf)
print("Random Forest Confusion Matrix:\n", cm_rf)
print("\nClassification Report (Random Forest):\n")
print(classification_report(y_test, y_pred_rf))


## 6. 模型错误类型简单解释（False Positives & False Negatives）

在上面的混淆矩阵中：
- **True Positive (TP)**：真实是 Hit(1)，模型也预测为 Hit(1)
- **True Negative (TN)**：真实是 Not Hit(0)，模型也预测为 Not Hit(0)
- **False Positive (FP)**：真实是 Not Hit(0)，但模型预测成 Hit(1)
- **False Negative (FN)**：真实是 Hit(1)，但模型预测成 Not Hit(0)

在本任务场景下可以这样理解：
- **FP（误判成 Hit 的歌）**：模型把一些其实播放量没那么高的歌，当成了热门歌曲。
- **FN（漏掉的 Hit 歌）**：模型把一些真实热门歌曲当成了普通歌。

通常来说：
- 如果你更关心“不要错过真正有潜力的热门歌”，就要尽量减少 **FN**。
- 如果你更关心“不要推荐太多其实不火的歌”，就要尽量减少 **FP**。

你可以在报告或 PPT 中，结合 Random Forest 的混淆矩阵，简单说明：
哪个错误（FP/FN）更多，以及这在业务上的含义。

## 7. 简单预测演示（Prediction Demo）
为了满足作业中 **PREDICT / Prediction Demo** 的要求，这里做一个非常简单的演示：

1. 取测试集中的前 10 条样本
2. 使用上面训练好的 **Random Forest 模型** 进行预测
3. 对比真实标签 (`y_true`) 和预测标签 (`y_pred`)


In [ ]:

# 从测试集中取前 10 条样本
sample_X = X_test.head(10)
sample_y_true = y_test.head(10)

# 使用随机森林模型进行预测
sample_y_pred = rf_clf.predict(sample_X)

# 整理成一个小表格方便查看
demo_df = sample_X.copy()
demo_df['y_true'] = sample_y_true.values
demo_df['y_pred'] = sample_y_pred

demo_df


## 简单总结
- 本 Notebook 展示了一个从 **Spotify 歌曲数据 → 构造 Hit 标签 → 训练分类模型** 的完整流程。
- 你可以在此基础上：
  - 增加更多可视化（比如散点图、直方图、热力图）
  - 尝试更多特征工程
  - 调参（超参数搜索）
  - 保存你表现最好的模型，用于 Part 3 的预测演示。